In [1]:
import findspark
findspark.init()

# Spark function
from pyspark import Row, StorageLevel
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import udf, explode, array, when
from pyspark.sql.types import IntegerType, StringType, ArrayType, BooleanType, MapType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Python function
import re
import subprocess
from functools import reduce 
import copy
import operator
import itertools

appname = input("appname, folder name : ")
folder_name = copy.deepcopy(appname) 
gvcf_count = int(input("gvcf count : "))

# Start for Spark Session
spark = SparkSession.builder.master("spark://master:7077")\
                        .appName(appname)\
                        .config("spark.driver.memory", "8G")\
                        .config("spark.driver.maxResultSize", "5G")\
                        .config("spark.executor.memory", "25G")\
                        .config("spark.memory.fraction", 0.1)\
                        .config("spark.sql.shuffle.partitions", 100)\
                        .config("spark.eventLog.enabled", "true")\
                        .config("spark.cleaner.periodicGC.interval", "15min")\
                        .getOrCreate()

appname, folder name : gvcf_40_0409
gvcf count : 40


## Function

In [2]:
def hadoop_list(length, hdfs):
    args = "hdfs dfs -ls "+ hdfs +" | awk '{print $8}'"
    proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    s_output, s_err = proc.communicate()
    all_dart_dirs = s_output.split()
    return all_dart_dirs[:length]

def headerVCF(hdfs, spark):
    col_name = ["key", "value"]
    vcf = spark.sparkContext.textFile(hdfs)
    vcf = vcf.filter(lambda x : re.match("^##", x))\
             .map(lambda x : x.split("=", 1))\
             .toDF(col_name)
    return vcf

def select_list(value, index):
    return value[index]
select_list = udf(select_list, StringType())

#.withColumn("ALT", F.regexp_replace(F.col("ALT"), "(<NON_REF>|,<NON_REF>)", ""))\
def preVCF(hdfs, spark):
    vcf = spark.sparkContext.textFile(hdfs)
    col_name = vcf.filter(lambda x : x.startswith("#CHROM")).first().split("\t")
    vcf_data = vcf.filter(lambda x : re.match("[^#][^#]", x))\
                       .map(lambda x : x.split("\t"))\
                       .toDF(col_name)\
                       .withColumn("POS", F.col("POS").cast(IntegerType()))\
                       .withColumn("ALT", F.array_remove(F.split(F.col("ALT"), ","), "<NON_REF>"))\
                       .withColumn("FORMAT", F.array_remove(F.split(F.col("FORMAT"), ":"), "GT"))\
                       .withColumn("INFO", when(F.col("INFO").startswith("END="), None).otherwise(F.col("INFO")))
    
    sample_name = vcf_data.columns[-1]
    vcf_data = vcf_data.drop("QUAL", "FILTER", sample_name)
    
    for index in range(len(vcf_data.columns)):
        compared_arr = ["#CHROM", "POS", "REF"]
        if vcf_data.columns[index] not in compared_arr:
            if vcf_data.columns[index] != "ALT":
                vcf_data = vcf_data.withColumn(vcf_data.columns[index], F.array(vcf_data.columns[index]))
            vcf_data = vcf_data.withColumnRenamed(vcf_data.columns[index], vcf_data.columns[index] + "_" + sample_name)     
            
    vcf_data = vcf_data.withColumnRenamed("REF", "REF_" + sample_name)    
    vcf_data = vcf_data.withColumn("count", F.length(vcf_data.columns[3]))
    
    # window & split col parameter 
    sample_ref = vcf_data.columns[3]
    indel_window = Window.partitionBy("#CHROM").orderBy("POS")
    split_col = F.split("REF_temp", '_')
    ###
    
    vcf_data = vcf_data.withColumn("count", when(F.col("count") >= 2, F.lead("POS", 1).over(indel_window) - F.col("POS") - F.lit(1))\
                                           .otherwise(F.lit(0)))

    not_indel = vcf_data.drop("count").withColumn(sample_ref, F.array(F.col(sample_ref)))
    indel = vcf_data.filter(F.col("count") >= 1)\
                .withColumn(sample_ref, ref_melt(F.col(sample_ref), F.col("count"))).drop("count")\
                .withColumn(sample_ref, explode(F.col(sample_ref))).withColumnRenamed(sample_ref, "REF_temp")\
                .withColumn(sample_ref, F.array(split_col.getItem(0))).withColumn('POS_var', split_col.getItem(1))\
                .drop(F.col("REF_temp")).withColumn("POS", (F.col("POS") + F.col("POS_var")).cast(IntegerType()))\
                .drop(F.col("POS_var"))\
                .withColumn(vcf_data.columns[2], F.array(F.lit(".")))\
                .withColumn(vcf_data.columns[4], F.array(F.lit("*")))
    
    return not_indel.unionByName(indel) 

def sampleVCF(hdfs, spark):
    vcf = spark.sparkContext.textFile(hdfs)
    col_name = vcf.filter(lambda x : x.startswith("#CHROM")).first().split("\t")
    vcf_data = vcf.filter(lambda x : re.match("[^#][^#]", x))\
                       .map(lambda x : x.split("\t"))\
                       .toDF(col_name)\
                       .withColumn("POS", F.col("POS").cast(IntegerType()))
    
    sample_name = vcf_data.columns[-1]
    vcf_data = vcf_data.withColumnRenamed("FORMAT", "FORMAT_" + sample_name)\
                       .withColumn(sample_name, F.array_join(F.slice(F.split(F.col(sample_name), ":"), 2, 8), ":"))
    return vcf_data.select(F.col("#CHROM").alias("CHROM"), F.col("POS"), 
                           F.array(F.col("FORMAT_"+sample_name), F.col("ALT"), F.col(sample_name)).alias(sample_name))

#.select(F.col("#CHROM").alias("CHROM"), F.col("POS"), 
#                           F.concat_ws("&",F.col("ALT"), F.col("FORMAT"), F.col(sample_name)).alias(sample_name))

def gatkVCF(hdfs, spark):
    vcf = spark.sparkContext.textFile(hdfs)
    #header_contig = vcf.filter(lambda x : re.match("^#", x))
    col_name = vcf.filter(lambda x : x.startswith("#CHROM")).first().split("\t")
    vcf_data = vcf.filter(lambda x : re.match("[^#][^#]", x))\
                       .map(lambda x : x.split("\t"))\
                       .toDF(col_name)
    return vcf_data

def chunks(lst, n):
    for index in range(0, len(lst), n):
        yield lst[index:index + n]

def unionAll(*dfs):
    return reduce(DataFrame.unionByName, dfs) 

def ref_melt(ref, count):
    temp = list(ref)[1:count + 1]
    return_str = []
    for num in range(len(temp)):
        return_str.append(temp[num] + "_" + str(int(num + 1)))
    return return_str
ref_melt = udf(ref_melt, ArrayType(StringType()))


def reduce_join(left, right):   
    return_vcf = left.join(right, ["#CHROM", "POS"], "full")

    ###
    remove_colname = right.columns[2:]
    l_name = left.columns
    r_name = right.columns
    v_name = return_vcf.columns
    name_list = ["REF", "ID", "ALT", "INFO", "FORMAT"]
    
    for name in name_list:
        if name == "INFO":
            return_vcf = return_vcf.withColumn(column_name(l_name, name)[0], 
                                       when(F.isnull(column_name(l_name, name)[0]), F.col(column_name(r_name, name)[0]))\
                                       .when(F.isnull(column_name(r_name, name)[0]), F.col(column_name(l_name, name)[0]))
                                       .otherwise(F.array_union(*column_name(v_name, name))))
        else :
            return_vcf = return_vcf.withColumn(column_name(l_name, name)[0], 
                                           when(F.isnull(column_name(l_name, name)[0]), F.col(column_name(r_name, name)[0]))\
                                           .when(F.isnull(column_name(r_name, name)[0]), F.col(column_name(l_name, name)[0]))
                                           .otherwise(F.array_union(*column_name(v_name, name))))
    return_vcf = return_vcf.drop(*remove_colname)
                                        
    return return_vcf

def column_rename(vcf):
    name_list = ["REF", "ID", "ALT", "INFO", "FORMAT"]
    for name in name_list:
        vcf = vcf.withColumnRenamed(column_name(vcf.columns, name)[0], name)
    return vcf
    
def reduce_inner_join(left, right):   
    return_vcf = left.join(right, ["#CHROM", "POS"], "inner")
    return return_vcf

def column_name(df_col, name):
    return_list = []
    for col in df_col:
        if col.startswith(name):
            return_list.append(col)
    return return_list

def column_revalue(vcf):
    # info 값 수정 필요
    name_list = ["ID", "REF","ALT", "INFO", "FORMAT"]
    for name in name_list:
        if name == "FORMAT":
            vcf = vcf.withColumn(name, F.array_sort(F.array_distinct(F.flatten(F.col(name)))))
            vcf = vcf.withColumn(name, F.concat(F.lit("GT:"), F.array_join(F.col(name), ":")))
        elif name == "ALT":
            vcf = vcf.withColumn(name, F.array_join(F.concat(F.col(name), F.array(F.lit("<NON_REF>"))), ","))
        else:
            vcf = vcf.withColumn(name, F.array_max(F.col(name)))
            
    return vcf

# 중복찾기
def find_duplicate(temp):
    return temp.groupBy(F.col("#CHROM"), F.col("POS")).agg((F.count("*")>1).cast("int").alias("e"))\
               .orderBy(F.col("e"), ascending=False)
# 10개 기준임.
def join_split(v_list):
    stage1_list = list(chunks(v_list, 5))
    stage1 = []
    for vcf in stage1_list:
        stage1.append(reduce(reduce_join, vcf))
    return reduce(reduce_join, stage1)

def join_inner(v_list):
    if len(v_list) <= 1:
        return v_list[0]
    else :
        return v_list[0].join(v_list[1], ["#CHROM", "POS"], "inner")

def join_split_inner(v_list, num):
    stage1_list = list(chunks(v_list, num))
    stage1 = []
    for vcf in stage1_list:
        temp = list(chunks(vcf, 2))
        temp = list(map(join_inner, temp))
        stage1.append(reduce(reduce_inner_join, temp))
    return stage1

def join_split_inner_2(v_list, num):
    stage1_list = list(chunks(v_list, num))
    stage1 = []
    for vcf in stage1_list:
        stage1.append(reduce(reduce_inner_join, vcf))
    return stage1

def vcf_join(v_list):
    chunks_list = list(chunks(v_list, 10))
    map_list = list(map(join_split, chunks_list))
    if len(map_list) <= 1:
        return map_list[0]
    elif len(map_list) == 2:
        return reduce(reduce_join, map_list)
    else:
        flag = True
        while flag == True:
            if len(map_list) == 2:
                flag = False
            else :
                map_list = list(chunks(map_list, 2))
                map_list = list(map(join_split, map_list))
        return reduce(reduce_join, map_list)    

def headerWrite(vcf, vcf_list, index, spark):
    contig = vcf_list[index].toPandas()
    contig_dup = contig["key"].drop_duplicates().tolist()
    contig_len = list(range(len(contig_dup)))
    contig_DF = spark.createDataFrame(list(zip(contig_dup, contig_len)), ["key", "index"])

    return vcf.dropDuplicates(["value"]).join(contig_DF, ["key"], "inner")\
              .select(F.concat_ws("=", F.col("key"), F.col("value")).alias("meta"))\
              .coalesce(1)

def parquet_revalue(vcf, indel_com):
    temp = indel_com.join(vcf, ["CHROM", "POS"], "full")
    sample_name = temp.columns[-1]
    
    sample_w = Window.partitionBy(F.col("CHROM")).orderBy(F.col("POS")).rangeBetween(Window.unboundedPreceding, Window.currentRow)
    temp = temp.withColumn(sample_name, F.last(sample_name, ignorenulls=True).over(sample_w))
    
    
    # scala UDF
    null_not_value = temp.filter(F.col("FORMAT") != F.element_at(F.col(sample_name), 1))\
                         .selectExpr("CHROM", "POS","revaluationPL(FORMAT, value, {}) as {}".format(sample_name, sample_name))
        
    null_value = temp.filter(F.col("FORMAT") == F.element_at(F.col(sample_name), 1)).drop("FORMAT", "value")\
                     .withColumn(sample_name, F.concat(F.lit("./.:"), F.element_at(F.col(sample_name), 3)))
    
    value_union = null_not_value.union(null_value).withColumnRenamed("CHROM", "#CHROM")
    return value_union.repartition(F.col("#CHROM"))

def parquet_revalue_not_union(vcf, indel_com):
    temp = indel_com.join(vcf, ["CHROM", "POS"], "full")
    sample_name = temp.columns[-1]
    
    sample_w = Window.partitionBy(F.col("CHROM")).orderBy(F.col("POS")).rangeBetween(Window.unboundedPreceding, Window.currentRow)
    temp = temp.withColumn(sample_name, F.last(sample_name, ignorenulls=True).over(sample_w))
    
    # scala UDF
    null_not_value = temp.withColumn(sample_name, when(F.col("FORMAT") != F.element_at(F.col(sample_name), 1), 
                                                       F.expr("revaluationPL(FORMAT, value, {})".format(sample_name)))\
                                                  .otherwise(F.concat(F.lit("./.:"), F.element_at(F.col(sample_name), 3))))\
                         .withColumnRenamed("CHROM", "#CHROM")
    return null_not_value

#.drop("FORMAT", "value")
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################

## Parquet write for info filed

In [ ]:
# vcf load
hdfs = "hdfs://master:9000"def parquet_revalue_del_2(vcf, indel_com):
    temp = indel_com.join(vcf, ["CHROM", "POS"], "full")
    sample_name = temp.columns[-1]
    
    sample_w = Window.partitionBy(F.col("CHROM")).orderBy(F.col("POS")).rangeBetween(Window.unboundedPreceding, Window.currentRow)  
    temp = temp.withColumn(sample_name, F.last(sample_name, ignorenulls=True).over(sample_w))
    
    # scala UDF
    null_not_value = temp.filter(F.map_keys(F.col(sample_name)) != F.col("FORMAT"))\
                         .selectExpr("CHROM", "POS","revaluationPL2({}, FORMAT, value) as {}".format(sample_name, sample_name))\
                         .withColumn(sample_name,  F.concat(F.lit("./.:"), F.array_join(F.col(sample_name), ":")))
    
    null_value = temp.filter(F.map_keys(F.col(sample_name)) == F.col("FORMAT")).drop("FORMAT", "value")\
                     .withColumn(sample_name, F.concat(F.lit("./.:"), F.array_join(F.slice(F.map_values(F.col(sample_name)), 1, 6), ":")))
    
    value_union = null_not_value.union(null_value).withColumnRenamed("CHROM", "#CHROM")
    return value_union

def parquet_revalue_del_1(vcf, indel_com):
    temp = indel_com.join(vcf, ["CHROM", "POS"], "full")
    sample_name = temp.columns[-1]
    
    sample_w = Window.partitionBy(F.col("CHROM")).orderBy(F.col("POS")).rangeBetween(Window.unboundedPreceding, Window.currentRow)  
    temp = temp.withColumn(sample_name, F.last(sample_name, ignorenulls=True).over(sample_w))\
               .selectExpr("CHROM", "POS","revaluationPL({}, FORMAT, value) as {}".format(sample_name, sample_name))\
               .withColumn(sample_name,  F.concat(F.lit("./.:"), F.array_join(F.col(sample_name), ":")))
    
    return temp.withColumnRenamed("CHROM", "#CHROM")
hdfs_list = hadoop_list(gvcf_count, "/raw_data/gvcf")
vcf_header = list()
vcf_list = list()

for index in range(len(hdfs_list)):
    vcf_header.append(headerVCF(hdfs + hdfs_list[index].decode("UTF-8"), spark))
    vcf_list.append(preVCF(hdfs + hdfs_list[index].decode("UTF-8"), spark).persist(StorageLevel.MEMORY_ONLY))
header = unionAll(*vcf_header).persist(StorageLevel.MEMORY_ONLY)
header.count()
    
headerWrite(header, vcf_header, 0, spark).write.format("text").option("header", "false").mode("append")\
                                  .save("/raw_data/output/gvcf_output/"+ folder_name + "//" + "gvcf_meta.txt")

info_last = header.filter(F.col("key") == "##contig").select("value").dropDuplicates(["value"])\
      .withColumn("value", F.split(F.col("value"), ","))\
      .select(F.regexp_replace(select_list(F.col("value"), F.lit(0)), "<ID=", "").alias("#CHROM"), 
              F.concat(F.lit("END="), F.regexp_replace(select_list(F.col("value"), F.lit(1)), "length=", "")).alias("INFO"))

vcf = column_rename(vcf_join(vcf_list))
vcf = column_revalue(vcf).persist(StorageLevel.MEMORY_ONLY)
vcf.count()

#window
# isNull() ---> isnull()
info_window = Window.partitionBy("#CHROM").orderBy("POS")
vcf_not_indel = vcf.withColumn("INFO", when(F.col("INFO").isNull(), F.concat(F.lit("END="), F.lead("POS", 1).over(info_window) - 1))\
                              .otherwise(F.col("INFO")))

not_last = vcf_not_indel.filter(F.col("INFO").isNotNull())
last = vcf_not_indel.filter(F.col("INFO").isNull())\
                 .drop("INFO").join(info_last, ["#CHROM"], "inner")\
                 .select("#CHROM", "POS", "ID", "REF", "ALT", "INFO", "FORMAT")

unionAll(*[not_last, last])\
                 .orderBy(F.col("#CHROM"), F.col("POS"))\
                 .dropDuplicates(["#CHROM", "POS"])\
                 .write.mode('overwrite')\
                 .parquet("/raw_data/output/gvcf_output/" + folder_name + "//info.g.vcf")

spark.catalog.clearCache()
spark.stop()

## Parquet write for sample

In [4]:
cnt = 0
num = 10
part_num = 80
# Start for Spark Session with write
spark = SparkSession.builder.master("spark://master:7077")\
                        .appName(appname + "_sample" + str(num) + "_" + str(part_num))\
                        .config("spark.driver.memory", "8G")\
                        .config("spark.driver.maxResultSize", "5G")\
                        .config("spark.executor.memory", "25G")\
                        .config("spark.sql.shuffle.partitions", part_num)\
                        .config("spark.eventLog.enabled", "true")\
                        .config("spark.memory.fraction", 0.05)\
                        .config("spark.cleaner.periodicGC.interval", "10min")\
                        .getOrCreate()
# scala UDF
spark.udf.registerJavaFunction("revaluationPL", "scalaUDF.revaluationPL3", StringType())
#spark.udf.registerJavaFunction("revaluationPL", "scalaUDF.revaluationPL", ArrayType(StringType()))
#spark.udf.registerJavaFunction("refalt2array", "scalaUDF.refalt2array", ArrayType(StringType()))

In [ ]:
# sample parquet write
hdfs = "hdfs://master:9000"
hdfs_list = hadoop_list(gvcf_count, "/raw_data/gvcf")
vcf_list = list()
for index in range(len(hdfs_list)):
    vcf_list.append(sampleVCF(hdfs + hdfs_list[index].decode("UTF-8"), spark))
    
indel_com = spark.read.parquet("/raw_data/output/gvcf_output/" + folder_name + "//info.g.vcf")\
                 .select(F.col("#CHROM").alias("CHROM"), "POS", "FORMAT", F.concat(F.array("REF"),F.split(F.col("ALT"), ",")).alias("value"))\
                 .orderBy(F.col("CHROM"), F.col("POS"))\
                 .persist(StorageLevel.MEMORY_ONLY)
indel_com.count()

In [8]:
parquet_list = list(map(lambda arg : parquet_revalue_not_union(arg, indel_com), vcf_list))
for parquet in join_split_inner(parquet_list, num):
    parquet.write.mode('overwrite')\
            .parquet("/raw_data/output/gvcf_output/"+ folder_name + "//" + "sample_" + str(cnt) + ".g.vcf")
    cnt += num
    
spark.catalog.clearCache()
spark.stop()

In [3]:
spark.catalog.clearCache()
spark.stop()

In [16]:
parquet_list = list(map(lambda arg : parquet_revalue_not_union(arg, indel_com), vcf_list))

In [17]:
parquet_list[0].show(4000,truncate=False)

+------+-----+--------------------------------+---------------------------+----------------------------------------------------+
|#CHROM|POS  |FORMAT                          |value                      |ND02798                                             |
+------+-----+--------------------------------+---------------------------+----------------------------------------------------+
|chr12 |1    |GT:DP:GQ:MIN_DP:PL              |[N, <NON_REF>]             |./.:0:0:0:0,0,0                                     |
|chr12 |60019|GT:DP:GQ:MIN_DP:PL              |[C, <NON_REF>]             |./.:0:0:0:0,0,0                                     |
|chr12 |60049|GT:DP:GQ:MIN_DP:PL              |[G, <NON_REF>]             |./.:0:0:0:0,0,0                                     |
|chr12 |60065|GT:DP:GQ:MIN_DP:PL              |[C, <NON_REF>]             |./.:0:0:0:0,0,0                                     |
|chr12 |60092|GT:DP:GQ:MIN_DP:PL              |[T, <NON_REF>]             |./.:0:0:0:0,0,0       